In [ ]:

데이터 불러오기부터
가. 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np

In [ ]:

# 나. 데이터 불러오기
df_feature = pd.read_csv("onenavi_train_feature.csv",sep="|")
df_target = pd.read_csv("onenavi_train_target.csv",sep="|")

In [ ]:
# 나. Train/Test Data Split
from sklearn.model_selection import train_test_split

# train_test_split
train_x, test_x, train_y, test_y = train_test_split(df_feature, df_target, test_size=0.20, random_state=42)

In [ ]:

1. 회귀문제에 딥러닝 모델 적용하기
딥러닝은 머신 러닝의 특정한 한 분야로서 연속된 층layer에서 점진적으로 의미 있는 표현을 배우는 데 강점이 있으며, 데이터로부터 표현을 학습하는 새로운 방식입니다.
즉 계층적 표현 학습이라고도 할 수 있습니다.
[공식 Document 및 참고자료]

텐서 플로우 '자동차 연비 예측하기: 회귀'(https://www.tensorflow.org/tutorials/keras/regression?hl=ko)
텐서 플로우 블로그(https://tensorflow.blog/)
가. 라이브러리 Import

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:

나. 모델링
[모델링에서 눈여겨 볼 것들!]

activation : output이 다음 레이어로 전해지는 과정 중 역할을 수행하는 수학적인 게이트(gate) image-2.png
optimizer : 손실함수를 최소화하도록 도와주는 알고리즘 image.png

In [ ]:
# train 데이터 셋을 확인해보겠습니다.
train_x

In [ ]:
# 모델 만들기 : 아주 간단한 모델

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_x.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  model.compile(loss='mse',
                optimizer='adam',
                metrics=['mae', 'mse'])
  return model

In [ ]:

만약에 분류문제를 해결해야 한다면!(아래 내용은 일반적으로 선호되는 내용이지 절대적이라고 하지는 않습니다.)
activation설정
마지막 출력층에 Label의 열이 하나고 두 개의 값으로 이루어진 이진분류라면 sigmoid
Label의 열이 두개 이상이라면 softmax
loss설정
출력층 activation이 sigmoid 인 경우: binary_crossentropy
출력층 activation이 softmax 인 경우:
원핫인코딩(O): categorical_crossentropy
원핫인코딩(X): sparse_categorical_crossentropy
metrics를 'acc' 혹은 'accuracy'로 지정하면, 학습시 정확도를 모니터링 할 수 있습니다.
원핫인코딩 여부 체크방법

In [ ]:
# Case1 : 원핫인코딩이 안된 데이터
temp_y=[9,5,6,7,1,2,5,4,6,0]
temp_y[0]

In [ ]:
# Case2 : 원핫인코딩 적용
print(tf.one_hot(temp_y[0], 10))
print(tf.one_hot(temp_y, 10))

In [ ]:

# 다시 본론으로 들어가 실습을 진행하겠습니다.
model = build_model()

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# 모델 학습
early_stopping = EarlyStopping(monitor='val_loss', patience=10) # 조기종료 콜백함수 정의

checkpoint_path = 'tmp_checkpoint.ckpt'
cb_checkpoint = ModelCheckpoint(checkpoint_path, save_weights_only=True, monitor='val_loss',
                               verbose=1, save_best_only=True) # 체크포인트 저장

history = model.fit(train_x, train_y, epochs=30,  
                   validation_data = (test_x,test_y),
                    callbacks=[cb_checkpoint, early_stopping]
                    )

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 30), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, 30), history.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 30), history.history["mae"], label="train_mae")
plt.plot(np.arange(0, 30), history.history["val_mae"], label="val_mae")
plt.title("Training mae")
plt.xlabel("Epoch #")
plt.ylabel("mae")
plt.legend()
plt.show()

In [ ]:
# 최적 모델 불러오기 및 저장
model.load_weights(checkpoint_path)
model.save("DeeplearningModel.h5")

In [ ]:

모델을 이해해봅시다
weights를 확인해서 가중치와 편향을 알 수 있습니다.
image.png [참고] 모두의 딥러닝

In [ ]:
# Input_layer 가중치
model.layers[0].get_weights()[0]

In [ ]:
# Input_layer 편향
model.layers[0].get_weights()[1]

In [ ]:
# Dense 가중치
model.layers[1].get_weights()[0]

In [ ]:
# Dense 편향
model.layers[1].get_weights()[1]

In [ ]:
# output 가중치
model.layers[2].get_weights()[0]

In [ ]:
# output 편향
model.layers[2].get_weights()[1]